In [1]:
import pandas as pd
import plotly.express as px
from google.colab import drive
from pprint import pprint, pformat
drive.mount('/content/modules', force_remount=True)

Mounted at /content/modules


In [ ]:
# Read IMDB data
db = pd.read_csv("/content/modules/My Drive/IMDB1.csv")
# Preliminary checkup
pprint(['shape', db.shape])
pprint(['columns', db.columns])
if "Revenue(Millions)" in db.columns: # Rename column for convenience
    db = db.rename(columns={"Revenue(Millions)": "Revenue"})
pprint(['dtypes', db.dtypes])
db.info()
categorical_columns = ["Title", "Genre", "Description", "Director", "Actors"]
for col in categorical_columns:
    pprint([col, db[col].nunique(), db[col].dropna().unique()[:3]])
numeric_columns = ["Year", "Runtime", "Votes", "Rating", "Revenue", "Metascore"]
pprint(db[numeric_columns].describe())
pprint(db.isnull().sum())
#db.head()

In [ ]:
# Convert comma-separated lists into Python lists
re_name = r'[ ]*([^,]+?)[ ]*(?:,|$)'
db.Genre    = db.Genre.str.findall(re_name)
db.Actors   = db.Actors.str.findall(re_name)
db.Director = db.Director.str.findall(re_name)
db.head()

In [ ]:
genres    = db[['Title', 'Genre']].explode('Genre')
actors    = db[['Title', 'Actors']].explode('Actors')
directors = db[['Title', 'Director']].explode('Director')
pprint(genres['Genre'].value_counts())
pprint(actors['Actors'].value_counts())
pprint(directors['Director'].value_counts())

In [ ]:
genres = db[['Title', 'Genre', 'Rating', 'Revenue', 'Votes', 'Runtime', 'Year']].explode('Genre')
genres_agg = genres.groupby('Genre').agg(
    num_movies=('Title', 'nunique'),
    avg_rating=('Rating', 'mean'),
    total_revenue=('Revenue', 'sum'),
    avg_runtime=('Runtime', 'mean'),
    total_votes=('Votes', 'sum'),
    earliest_year=('Year', 'min'),
    latest_year=('Year', 'max')
).reset_index()
pprint(genres_agg.sort_values(by='num_movies', ascending=False))

In [ ]:
directors = db[['Title', 'Director', 'Year', 'Rating', 'Revenue', 'Votes', 'Runtime']].explode('Director')
director_year_rating = pd.pivot_table( # Average movie rating per Director by Year.
    directors,
    index='Director',
    columns='Year',
    values='Rating',
    aggfunc='mean',
    fill_value=0
)
pprint(director_year_rating.head(10))

In [ ]:
directors = db[['Title', 'Director', 'Year', 'Rating', 'Revenue', 'Votes', 'Runtime']].explode('Director')
director_summary = directors.groupby('Director').agg(
    num_movies=('Title', 'nunique'),
    avg_rating=('Rating', 'mean'),
    total_revenue=('Revenue', 'sum'),
    avg_votes=('Votes', 'mean'),
    avg_runtime=('Runtime', 'mean')
).reset_index()
pprint(director_summary.sort_values(by='num_movies', ascending=False).head(10))

In [ ]:
actors = db[['Title', 'Actors', 'Genre']].explode('Actors')
actors = actors.explode('Genre')
actor_genre_pivot = pd.pivot_table(
    actors,
    index='Actors',
    columns='Genre',
    values='Title',
    aggfunc='count',
    fill_value=0
)
pprint(actor_genre_pivot.head(10))